# Operando no MetaTrader e comprando as ações indicadas pelo modelo.

## Coleta de dados das empresas

In [9]:
import pandas as pd
import requests
import telebot
import MetaTrader5 as mt5

In [2]:
url = 'http://www.fundamentus.com.br/resultado.php'
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}
r = requests.get(url, headers=header)
tabela = pd.read_html(r.text,  decimal=',', thousands='.')[0]

#TRATANDO DADOS
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  tabela[coluna] = tabela[coluna].str.replace('.', '')
  tabela[coluna] = tabela[coluna].str.replace(',', '.')
  tabela[coluna] = tabela[coluna].str.rstrip('%').astype('float') / 100

#CRIANDO RANKING
tabela = tabela[['Papel', 'Cotação', 'EV/EBIT', 'ROIC', 'Liq.2meses', 'P/L']]
tabela['Empresa'] = tabela['Papel'].str[:4]
tabela = tabela.drop_duplicates(subset='Empresa')
tabela = tabela.set_index('Papel')
tabela = tabela[tabela['Liq.2meses'] > 1000000]
tabela = tabela[tabela['P/L'] > 0]
tabela = tabela[tabela['EV/EBIT'] > 0]
tabela = tabela[tabela['ROIC'] > 0]
tabela



,Cotação,EV/EBIT,ROIC,Liq.2meses,P/L,Empresa
Papel,,,,,,
PETR4,31.94,2.43,0.2895,1.530180e+09,2.66,PETR
JALL3,8.41,27.53,0.0348,8.716290e+06,3.99,JALL
ALSO3,22.77,13.28,0.0324,1.125940e+08,4.08,ALSO
CMIG4,12.33,4.46,0.1635,1.040030e+08,4.10,CMIG
AURE3,13.61,16.85,0.0321,3.414810e+07,4.40,AURE
...,...,...,...,...,...,...
CEAB3,5.35,6.95,0.0528,2.621430e+07,56.21,CEAB
YDUQ3,20.37,14.10,0.0899,1.060570e+08,57.15,YDUQ
PETZ3,5.48,12.48,0.0631,1.103270e+08,61.61,PETZ


## Criação do Ranking de acordo com o método da Magic Formula

In [4]:
tabela['RANKING_EV/EBIT'] = tabela['EV/EBIT'].rank(ascending = True)
tabela['RANKING_ROIC'] = tabela['ROIC'].rank(ascending = False)
tabela['RANKING_TOTAL'] = tabela['RANKING_EV/EBIT'] + tabela['RANKING_ROIC']

tabela = tabela.sort_values('RANKING_TOTAL')
tabela = tabela.head(15)
tabela

,Cotação,EV/EBIT,ROIC,Liq.2meses,P/L,Empresa,RANKING_EV/EBIT,RANKING_ROIC,RANKING_TOTAL
Papel,,,,,,,,,
PSSA3,26.10,0.22,0.6076,5.817340e+07,9.04,PSSA,1.0,1.0,2.0
PETR4,31.94,2.43,0.2895,1.530180e+09,2.66,PETR,3.0,6.0,9.0
WIZC3,5.91,2.20,0.2418,4.684090e+06,9.00,WIZC,2.0,12.0,14.0
KEPL3,11.22,4.98,0.4361,1.552060e+07,6.44,KEPL,12.0,2.0,14.0
CMIN3,4.28,4.82,0.2849,2.629940e+07,9.80,CMIN,10.0,7.0,17.0
FESA4,46.28,4.43,0.2557,9.665660e+06,5.99,FESA,7.0,10.0,17.0
VLID3,17.01,4.17,0.2064,5.854680e+06,8.50,VLID,5.0,16.0,21.0
TASA4,14.86,5.04,0.2433,8.655750e+06,6.10,TASA,13.0,11.0,24.0
PLPL3,10.43,5.79,0.3059,1.056890e+07,10.54,PLPL,21.5,4.0,25.5


## Envio de mensagem para o Telegram com as ações da Magic Formula

In [8]:
ranking = tabela.index
ranking = '\n'.join(f'{i+1}. {acao}' for i, acao in enumerate(ranking))
mensagem = f'RANKING DA MAGIC FORMULA:\n{ranking}'
print(mensagem)
bot = telebot.TeleBot("YOUR BOT HERE")
group = "YOUR GROUP HERE"
bot.send_message(group, mensagem)

RANKING DA MAGIC FORMULA:
1. PSSA3
2. PETR4
3. WIZC3
4. KEPL3
5. CMIN3
6. FESA4
7. VLID3
8. TASA4
9. PLPL3
10. GGBR3
11. PRNR3
12. AURA33
13. CMIG4
14. BEEF3
15. SAPR3


## Inicializar MetaTrader 5

Documentação: https://www.mql5.com/en/docs/python_metatrader5

In [10]:
mt5.initialize()

True

## Configurar o pacote das ações e enviar as ordens no MT5

In [11]:
tickers = tabela.index
tickers

Index(['PSSA3', 'PETR4', 'WIZC3', 'KEPL3', 'CMIN3', 'FESA4', 'VLID3', 'TASA4',
       'PLPL3', 'GGBR3', 'PRNR3', 'AURA33', 'CMIG4', 'BEEF3', 'SAPR3'],
      dtype='object', name='Papel')

In [30]:
for ticker in tickers:

    print(ticker)

    info_acoes = mt5.symbol_info(ticker)
    mt5.symbol_select(ticker)
    tick_min = mt5.symbol_info(ticker).point
    preco = mt5.symbol_info_tick(ticker).ask
    quantidade = 100.0
    ordem_compra = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": quantidade,
        "type": mt5.ORDER_TYPE_BUY,
        "price": preco,
        "magic": 1,
        "comment": "Fórmula Mágica",
        "type_time": mt5.ORDER_TIME_DAY,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result_compra = mt5.order_send(ordem_compra)
    print(result_compra)

PSSA3
OrderSendResult(retcode=10009, deal=1191625913, order=1613054176, volume=100.0, price=26.5, bid=26.48, ask=26.5, comment='Request executed', request_id=785341678, retcode_external=0, request=TradeRequest(action=1, magic=1, order=0, symbol='PSSA3', volume=100.0, price=26.5, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=0, type_filling=2, type_time=1, expiration=0, comment='Fórmula Mágica', position=0, position_by=0))
PETR4
OrderSendResult(retcode=10009, deal=1191625914, order=1613054177, volume=100.0, price=31.51, bid=31.5, ask=31.51, comment='Request executed', request_id=785341679, retcode_external=0, request=TradeRequest(action=1, magic=1, order=0, symbol='PETR4', volume=100.0, price=31.51, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=0, type_filling=2, type_time=1, expiration=0, comment='Fórmula Mágica', position=0, position_by=0))
UNIP3
OrderSendResult(retcode=10009, deal=1191625915, order=1613054178, volume=100.0, price=71.06, bid=70.62, ask=71.06, comment='Request ex